In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
input_size = (299, 299)  # Input size for InceptionV3 model
batch_size = 32  # Number of images to process in each batch

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:
train_data = data_generator.flow_from_directory(
    '../content/drive/MyDrive/MLProject/brain_tumor_dataset',
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_data = data_generator.flow_from_directory(
    '../content/drive/MyDrive/MLProject/brain_tumor_dataset',
    target_size=input_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 253 images belonging to 2 classes.
Found 253 images belonging to 2 classes.


In [ ]:
# Load the InceptionV3 model (pretrained on ImageNet)
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(input_size[0], input_size[1], 3)
)

In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    train_data,
    validation_data=validation_data,
    epochs=10
)

Epoch 1/10
8/8 [==============================] - 201s 24s/step - loss: 0.9674 - accuracy: 0.6640 - val_loss: 0.6869 - val_accuracy: 0.7075
Epoch 2/10
8/8 [==============================] - 152s 20s/step - loss: 0.4060 - accuracy: 0.8063 - val_loss: 0.4992 - val_accuracy: 0.7905
Epoch 3/10
8/8 [==============================] - 151s 20s/step - loss: 0.4238 - accuracy: 0.8261 - val_loss: 0.4632 - val_accuracy: 0.8063
Epoch 4/10
8/8 [==============================] - 155s 21s/step - loss: 0.3364 - accuracy: 0.8696 - val_loss: 0.2608 - val_accuracy: 0.8893
Epoch 5/10
8/8 [==============================] - 152s 20s/step - loss: 0.2924 - accuracy: 0.8893 - val_loss: 0.2715 - val_accuracy: 0.8696
Epoch 6/10
8/8 [==============================] - 152s 20s/step - loss: 0.2718 - accuracy: 0.8893 - val_loss: 0.2526 - val_accuracy: 0.9012
Epoch 7/10
8/8 [==============================] - 150s 20s/step - loss: 0.2499 - accuracy: 0.8893 - val_loss: 0.2144 - val_accuracy: 0.9170
Epoch 8/10
8/8 [====